In [135]:
!pip install beautifulsoup4 requests
!pip install selenium
!pip install webdriver-manager
!pip install summa
!pip install git+https://github.com/lovit/textrank.git
!pip install konlpy

In [196]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

#웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

def makeUrl(search, page, page2):
    search_urls = []
    base_url = "https://search.naver.com/search.naver?where=news&query="
    search = urllib.parse.quote(search)
    for p in range(page, page2 + 1):
        start = (p - 1) * 10 + 1
        search_url = base_url + search + "&start=" + str(start)
        search_urls.append(search_url)
    return search_urls
#(부동산 건축물대장 사기피해,부동산 등기부등본 사기피해, 부동산 신분증 사기피해,부동산 위임장 사기피해, 후견등기사항 부존재증명서 피해)
#검색어 입력
search = input("검색할 키워드를 입력해주세요:")

#검색 시작할 페이지 입력
page = int(1)
#page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):"))
print("\n크롤링할 시작 페이지: ",page,"페이지")   
#검색 종료할 페이지 입력
page2 = int(2)
#page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):"))
print("\n크롤링할 종료 페이지: ",page2,"페이지")   

# naver url 생성
search_urls = makeUrl(search,page,page2)

## selenium으로 navernews만 뽑아오기##
# 버전에 상관 없이 os에 설치된 크롬 브라우저 사용
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.implicitly_wait(3)

# selenium으로 검색 페이지 불러오기 #
naver_urls=[]

for i in search_urls:
    driver.get(i)
    time.sleep(1) #대기시간 변경 가능

    # 네이버 기사 눌러서 제목 및 본문 가져오기#
    # 네이버 기사가 있는 기사 css selector 모아오기
    a = driver.find_elements(By.CSS_SELECTOR,'a.info')

    # 위에서 생성한 css selector list 하나씩 클릭하여 본문 url얻기
    for i in a:
        i.click()

        # 현재탭에 접근
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(3) #대기시간 변경 가능

        # 네이버 뉴스 url만 가져오기
        url = driver.current_url
        print(url)

        if "news.naver.com" in url:
            naver_urls.append(url)
        else:
            pass
        
        # 현재 탭 닫기
        driver.close()

        # 다시처음 탭으로 돌아가기(매우 중요!!!)
        driver.switch_to.window(driver.window_handles[0])

print(naver_urls)



크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지:  2 페이지


C:\Users\smhrd\AppData\Local\Temp\ipykernel_20748\1821716915.py:42: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


https://www.segye.com/
https://n.news.naver.com/mnews/article/022/0003329149?sid=102
https://www.lawtimes.co.kr/
https://www.lawtimes.co.kr/
['https://n.news.naver.com/mnews/article/022/0003329149?sid=102']


In [197]:
###naver 기사 본문 및 제목 가져오기###
import re
# ConnectionError방지
headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36" }

titles = []
contents=[]
for i in naver_urls:
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")
    # 검색결과확인시
    #print(html)
    
    #뉴스 제목 가져오기
    title = html.select_one("div#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    # list합치기
    title = ''.join(str(title))
    # html태그제거
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1,repl='',string=title)
    titles.append(title)

    #뉴스 본문 가져오기
    content = html.select_one("div#dic_area")

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))
    
    #html태그제거 및 텍스트 다듬기
    content = re.sub(pattern=pattern1,repl='',string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2,'')

    contents.append(content)

print(titles)
print(contents)

['2019년 새해 이렇게 달라집니다']
['\n최저임금 시간당 8350원… 종부세 최고세율 3.2%로 상향\n\n\n\n내년부터 최저임금이 시간당 8350원으로 인상된다. 모든 6세 미만 아동에게 월 10만원의 아동수당이 지급되며, 난임 부부를 위한 시술비 지원 및 대상도 확대된다. 주택분 종합부동산세 최고세율은 노무현정부 수준인 3.2%로 인상된다. 장병 3만6500명에게는 패딩형 운동복이 보급된다. 26일 정부는 이 같은 내용을 담은 ‘새해 달라지는 정책과 제도’를 소개했다.소상공인 등 카드수수료 부담 완화△주택 종부세 최고세율 상향=주택분 종부세 최고세율이 현행 2%에서 3.2%로 상향된다. 3주택 이상 보유자와 서울 등 집값이 급등한 조정대상지역 2주택 이상 보유자가 대상이다. 또 종부세 과표 3억∼6억원 구간을 신설해 세율을 0.7%로 0.2%포인트 인상한다.△근로장려금 지급총액 3배로 확대 =근로장려금의 지급 요건 완화로 적용 대상이 크게 늘어난다. 우선 단독가구의 연령 요건(30세 이상)이 폐지되고, 소득 요건도 낮아진다. 정부는 이에 따라 근로장려금 지급총액이 약 3배로 확대될 것으로 추산하고 있다.△노후 경유차 교체하면 개별소비세 감면=오래된 경유차를 폐차하고 승용차를 새로 사는 경우 143만원 한도에서 개별소비세 70를 감면받는다. 2008년 이전에 최초 등록한 경유 자동차를 올해 6월 30일 현재 등록·소유한 자가 지원 대상이다.△신용카드 매출세액 공제 한도 확대·우대 기한 연장=자영업자를 지원하기 위해 신용카드 매출세액 공제 한도가 현행 500만원에서 내년부터 1000만원으로 확대된다. 매출세액 우대공제율(2.6, 1.3) 적용기한은 올해 말에서 2021년 말까지로 3년 연장된다.△소상공인·자영업자 카드수수료 부담 완화=소상공인·자영업자 등 가맹점의 카드수수료 부담을 줄이기 위해 신용카드 및 체크카드 우대수수료율 적용구간이 연매출 5억원 이하에서 30억원 이하로 확대된다. △박물관·미술관 입장료도 신용카드 소득공제=박물관·미술관 입장료도

In [198]:
import os
import pandas as pd

# 저장할 폴더 경로 설정
save_dir = 'C:/Users/smhrd/Desktop/news/'

# 데이터프레임 생성
data = {'titles': titles, 'contents': contents}
df = pd.DataFrame(data)

# 각 행마다 파일로 저장
for i, row in df.iterrows():
    # 파일 이름 지정
    file_name = f"{search}{i}.txt"
    
    # 파일 경로 지정
    file_path = os.path.join(save_dir, file_name)
    
    # 파일 쓰기
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(f"{row['titles']}\n\n{row['contents']}")


In [199]:
import numpy as np
from konlpy.tag import Okt
from textrank import KeysentenceSummarizer

# 파일 읽기
results = []
for i in range(len(titles)):
    file_path = f"C:/Users/smhrd/Desktop/news/{search}{i}.txt"
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()
        results.append(text)

# NumPy 배열로 변환
results_array = np.array(results)

# 토큰화 함수 정의
def okt_tokenizer(sent):
    okt = Okt()
    words = okt.nouns(sent)
    return words

# KeysentenceSummarizer 생성
summarizer = KeysentenceSummarizer(tokenize=okt_tokenizer, min_sim=0.3, verbose=False)

# 요약 생성
summaries = []
for content in results_array:
    summary = summarizer.summarize([content], topk=3)
    summaries.append(summary)

# 요약 결과 출력
for i, summary in enumerate(summaries):
    print(f"Summary for text {i + 1}:")
    for sentence in summary:
        print(sentence)
    print()


Summary for text 1:
(0, 1.0, '2019년 새해 이렇게 달라집니다\n\n\n최저임금 시간당 8350원… 종부세 최고세율 3.2%로 상향\n\n\n\n내년부터 최저임금이 시간당 8350원으로 인상된다. 모든 6세 미만 아동에게 월 10만원의 아동수당이 지급되며, 난임 부부를 위한 시술비 지원 및 대상도 확대된다. 주택분 종합부동산세 최고세율은 노무현정부 수준인 3.2%로 인상된다. 장병 3만6500명에게는 패딩형 운동복이 보급된다. 26일 정부는 이 같은 내용을 담은 ‘새해 달라지는 정책과 제도’를 소개했다.소상공인 등 카드수수료 부담 완화△주택 종부세 최고세율 상향=주택분 종부세 최고세율이 현행 2%에서 3.2%로 상향된다. 3주택 이상 보유자와 서울 등 집값이 급등한 조정대상지역 2주택 이상 보유자가 대상이다. 또 종부세 과표 3억∼6억원 구간을 신설해 세율을 0.7%로 0.2%포인트 인상한다.△근로장려금 지급총액 3배로 확대 =근로장려금의 지급 요건 완화로 적용 대상이 크게 늘어난다. 우선 단독가구의 연령 요건(30세 이상)이 폐지되고, 소득 요건도 낮아진다. 정부는 이에 따라 근로장려금 지급총액이 약 3배로 확대될 것으로 추산하고 있다.△노후 경유차 교체하면 개별소비세 감면=오래된 경유차를 폐차하고 승용차를 새로 사는 경우 143만원 한도에서 개별소비세 70를 감면받는다. 2008년 이전에 최초 등록한 경유 자동차를 올해 6월 30일 현재 등록·소유한 자가 지원 대상이다.△신용카드 매출세액 공제 한도 확대·우대 기한 연장=자영업자를 지원하기 위해 신용카드 매출세액 공제 한도가 현행 500만원에서 내년부터 1000만원으로 확대된다. 매출세액 우대공제율(2.6, 1.3) 적용기한은 올해 말에서 2021년 말까지로 3년 연장된다.△소상공인·자영업자 카드수수료 부담 완화=소상공인·자영업자 등 가맹점의 카드수수료 부담을 줄이기 위해 신용카드 및 체크카드 우대수수료율 적용구간이 연매출 5억원 이하에서 30억원 이하로 확대된다. △박물관·미술관 

In [200]:
len(results)

1

In [179]:
print(search)

부동산 등기부등본 사기 피해


In [201]:
from summa.summarizer import summarize
#주어진 경로에서 파일을 읽어서 리스트에 저장합니다.
results1 = []
titles_length = len(results)

for i in range(titles_length):
    file_path = f"C:/Users/smhrd/Desktop/news/{search}{i}.txt"
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()
        results1.append(text)
        print(text)
#리스트에 저장된 각각의 파일에 대해서 요약을 생성하고 요약된 결과를 파일에 저장합니다.
for i, content in enumerate(results1):
    try:
        summary = summarize(content, ratio=0.5) # 50%로 요약 생성
        summary = summary[:300]  # 300 글자로 자르기
        summary_file_path = f"C:/Users/smhrd/Desktop/news/{search}{i}.txt"
        with open(summary_file_path, "w", encoding="utf-8") as f:
            f.write(summary)
        print(f"{search}Summary {i + 1} has been saved.")
    except Exception as e:
        print(f"Error summarizing text {i + 1}: {e}")



2019년 새해 이렇게 달라집니다


최저임금 시간당 8350원… 종부세 최고세율 3.2%로 상향



내년부터 최저임금이 시간당 8350원으로 인상된다. 모든 6세 미만 아동에게 월 10만원의 아동수당이 지급되며, 난임 부부를 위한 시술비 지원 및 대상도 확대된다. 주택분 종합부동산세 최고세율은 노무현정부 수준인 3.2%로 인상된다. 장병 3만6500명에게는 패딩형 운동복이 보급된다. 26일 정부는 이 같은 내용을 담은 ‘새해 달라지는 정책과 제도’를 소개했다.소상공인 등 카드수수료 부담 완화△주택 종부세 최고세율 상향=주택분 종부세 최고세율이 현행 2%에서 3.2%로 상향된다. 3주택 이상 보유자와 서울 등 집값이 급등한 조정대상지역 2주택 이상 보유자가 대상이다. 또 종부세 과표 3억∼6억원 구간을 신설해 세율을 0.7%로 0.2%포인트 인상한다.△근로장려금 지급총액 3배로 확대 =근로장려금의 지급 요건 완화로 적용 대상이 크게 늘어난다. 우선 단독가구의 연령 요건(30세 이상)이 폐지되고, 소득 요건도 낮아진다. 정부는 이에 따라 근로장려금 지급총액이 약 3배로 확대될 것으로 추산하고 있다.△노후 경유차 교체하면 개별소비세 감면=오래된 경유차를 폐차하고 승용차를 새로 사는 경우 143만원 한도에서 개별소비세 70를 감면받는다. 2008년 이전에 최초 등록한 경유 자동차를 올해 6월 30일 현재 등록·소유한 자가 지원 대상이다.△신용카드 매출세액 공제 한도 확대·우대 기한 연장=자영업자를 지원하기 위해 신용카드 매출세액 공제 한도가 현행 500만원에서 내년부터 1000만원으로 확대된다. 매출세액 우대공제율(2.6, 1.3) 적용기한은 올해 말에서 2021년 말까지로 3년 연장된다.△소상공인·자영업자 카드수수료 부담 완화=소상공인·자영업자 등 가맹점의 카드수수료 부담을 줄이기 위해 신용카드 및 체크카드 우대수수료율 적용구간이 연매출 5억원 이하에서 30억원 이하로 확대된다. △박물관·미술관 입장료도 신용카드 소득공제=박물관·미술관 입장료도 신용카드 등 사용